In [ ]:
import pdfplumber, os
import os.path as osp
from pdf_extract.config import global_config as glob
from importlib import reload

reload(glob)

: 

In [55]:
fname = 'CV3'

i, page_objects = 0, {}
# The open method returns an instance of the pdfplumber.PDF class.
with pdfplumber.open(osp.join(glob.UC_DATA_DIR, f"example_cvs/{fname}.pdf")) as pdf:
    while i < len(pdf.pages):
        page = pdf.pages[i]
        page_objects[str(i)] = page
        print(f"working on page {i}")
        #print(page.extract_text())
        i += 1

working on page 0
working on page 1
working on page 2


In [41]:
page_instance = page_objects['2']

In [56]:
#pdf.metadata

In [43]:
page_instance

<Page:3>

In [44]:
page_instance = pdf.pages[1]

In [57]:
#page_instance.extract_text()

In [20]:
#page_instance.extract_words()